# Method of Simulated Moments

## Introduction

The simulated method of moments (SMM) is some kind of a sledgehammer approach to estimate model parameters. It is a deviation from the general method of moments (GMM) which does not require that the function mapping parameters to moments has a closed-form solution. Thus, we want to minimize the distance between the actual moments and the simulated moments implied by the model parameters. Mathematically, we want to minimize

$$
    \min_\theta \; (m(\theta) - m^*)' W (m(\theta) - m^*)
$$

where $\theta$ is the parameter vector, $m(\theta)$ are the simulated moments implied by the parameters, $m^*$ are the moments from the data and $W$ is the weighting matrix.

## The weighting matrix

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import respy as rp
import matplotlib.pyplot as plt

from respy.pre_processing.model_processing import process_params_and_options

In [2]:
params, options = rp.get_example_model("kw_2000", with_data=False)

In [3]:
simulate = rp.get_simulate_func(params, options)

KeyboardInterrupt: 

In [6]:
%debug

> c:\tools\miniconda3\envs\respy\lib\site-packages\numpy\core\fromnumeric.py(90)_wrapreduction()
     88                 return reduction(axis=axis, out=out, **passkwargs)
     89 
---> 90     return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
     91 
     92 



ipdb>  u


> c:\tools\miniconda3\envs\respy\lib\site-packages\numpy\core\fromnumeric.py(2182)sum()
   2180 
   2181     return _wrapreduction(a, np.add, 'sum', axis, dtype, out, keepdims=keepdims,
-> 2182                           initial=initial, where=where)
   2183 
   2184 



ipdb>  u


> <__array_function__ internals>(6)sum()



...

ipdb>  shares


['0.6151', '0.3849']


ipdb>  params


category               name       
delta                  delta                                                      0.934
wage_white_collar      constant                                                   8.864
                       exp_school                                                0.0709
                       hs_graduate                                              -0.0123
                       co_graduate                                               0.0173
                                                            ...                        
initial_exp_school_10  black                                                       0.52
initial_exp_school_11  black                                                       0.08
maximum_exp            school_20      Maximum level of experience for education (opt...
observables            black_0                                                   0.6151
                       black_1                                                   0.38

ipdb>  q


## Estimation

In [2]:
params, options, df = rp.get_example_model("kw_94_one")

### 1. Define function to calculate moments.

In [3]:
def calc_moments(df):
    counts = (
        df.groupby(["Period", "Choice"]).count()
        .fillna(0)
        .iloc[:, 0]
        .unstack(1)
    )
    sum_ = counts.sum(axis=1)
    shares = counts.div(sum_, axis=0)
    
    return shares.to_numpy().flatten()

In [4]:
moments = calc_moments(df)

### 2. Choose a weighting matrix.

In [5]:
def bootstrap_weighting_matrix(df, calc_moments, n_bootstraps, n_individuals):
    identifiers = df.Identifier.unique()
    
    moments = []
    for _ in range(n_bootstraps):
        idents = np.random.choice(identifiers, size=n_individuals, replace=True)
        sample = df.loc[df.Identifier.isin(idents)]
        bootstrapped_moments = calc_moments(sample)
        moments.append(bootstrapped_moments)
    moments = np.column_stack(moments)
    
    standard_deviations = moments.std(axis=1)
    standard_deviations = np.where(
        standard_deviations == 0, 10, standard_deviations
    )
    
    weighting_matrix = np.diag(standard_deviations)
        
    return weighting_matrix

In [6]:
W = bootstrap_weighting_matrix(df, calc_moments, 10, 200)

### 3. Estimate.

In [7]:
smm = rp.get_smm_func(params, options, moments, calc_moments, W)

In [8]:
from estimagic.optimization.optimize import minimize


constr = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"},
    {"loc": "lagged_choice_1_edu", "type": "fixed"},
    {"loc": "initial_exp_edu", "type": "fixed"},
    {"loc": "maximum_exp", "type": "fixed"},
]
params["group"] = params.index.get_level_values('category')

In [9]:
results, params = minimize(
    smm,
    params,
    "scipy_L-BFGS-B",
    algo_options={"maxfun": 1},
    constraints=constr,
)

In [10]:
results

{'fun': 0.0,
 'jac': [0.0,
  0.0,
  0.0,
  0.0,
  2.1866006306702985,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'nfev': 630,
 'nit': 0,
 'status': 1,
 'message': b'ABNORMAL_TERMINATION_IN_LNSRCH',
 'x': [0.95,
  9.21,
  0.038,
  0.033,
  -0.0005,
  0.0,
  0.0,
  8.48,
  0.07,
  0.067,
  -0.001,
  0.022000000000000002,
  -0.0005,
  0.0,
  0.0,
  -4000.0,
  17750.0,
  0.2,
  0.0,
  0.25,
  0.0,
  0.0,
  1500.0,
  0.0,
  0.0,
  0.0,
  1500.0,
  1.0,
  1.0],
 'success': False,
 'hess_inv': <29x29 LbfgsInvHessProduct with dtype=float64>,
 'internal_x': [0.95,
  9.21,
  0.038,
  0.033,
  -0.0005,
  0.0,
  0.0,
  8.48,
  0.07,
  0.067,
  -0.001,
  0.022000000000000002,
  -0.0005,
  0.0,
  0.0,
  -4000.0,
  17750.0,
  0.2,
  0.0,
  0.25,
  0.0,
  0.0,
  1500.0,
  0.0,
  0.0,
  0.0,
  1500.0,
  1.0,
  1.0]}